In [ ]:
!pip install av

     |████████████████████████████████| 37.2MB 1.2MB/s 


In [ ]:
import torchvision
import cv2
video_path = '/content/test_video2.mp4'
_time = torchvision.io.read_video_timestamps(video_path)
time_stamps = _time[0]
frames = torchvision.io.read_video(video_path, start_pts=time_stamps[10], end_pts=time_stamps[100])

/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:117: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."


In [ ]:
%%time
frames = torchvision.io.read_video(video_path, start_pts=time_stamps[199])


/usr/local/lib/python3.6/dist-packages/torchvision/io/video.py:117: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  + "follow-up version. Please use pts_unit 'sec'."


CPU times: user 380 ms, sys: 7.96 ms, total: 388 ms
Wall time: 408 ms


In [ ]:
%%time
import cv2
vidcap = cv2.VideoCapture(video_path)
success,image = vidcap.read()
count = 0
l = []
while success:
  success,image = vidcap.read()
  l.append(image)

CPU times: user 2.88 s, sys: 138 ms, total: 3.02 s
Wall time: 1.72 s


In [ ]:
# reader = torchvision.io.VideoReader(video_path, "video")
# reader.seek(2.0)
# frame = next(reader)

In [ ]:
import torchvision
import torch
import math
from torch.utils.data import DataLoader
# from torch.utils.data import IterableDataset


import random
from itertools import chain, cycle


In [ ]:
class IterDataset(torch.utils.data.IterableDataset):
    def __init__(self, data):
        super(MyIterableDataset).__init__()
        #  assert end > start, "this example code only works with end >= start"
        self.data_list = data_list
        self.batch_size = batch_size

    def shuffle_data_list(self):
        return random.sample(self.data_list, len(self.data_list))


    def process_data(self, data_list, batch_size):
        for x in data:
            yield x
        
    def get_stream(self, data_list):
        return chain.from_iterable(map(self.process_data, cycle(data_list)))

    def get_streams(self):
        pass

    def __iter__(self):
        return iter(self.data)

def worker_init_fn(worker_id):
    worker_info = torch.utils.data.get_worker_info()
    dataset = worker_info.dataset  # the dataset copy in this worker process
    overall_start = dataset.start
    overall_end = dataset.end
    # configure the dataset to only process the split workload
    per_worker = int(math.ceil((overall_end - overall_start) / float(worker_info.num_workers)))
    worker_id = worker_info.id
    dataset.start = overall_start + worker_id * per_worker
    dataset.end = min(dataset.start + per_worker, overall_end)